In [ ]:
### Data Retrieval from yahoo finance

import yfinance as yahooFinance
import pandas as pd
import numpy as np
import datetime
 
def get_data(startDate, endDate, tickers):
    s1 = yahooFinance.Ticker(tickers[0]).history(
        start=startDate, end=endDate).reset_index()
    s2 = yahooFinance.Ticker(tickers[1]).history(
        start=startDate, end=endDate).reset_index()
    s3 = yahooFinance.Ticker(tickers[2]).history(
        start=startDate, end=endDate).reset_index()
    s4 = yahooFinance.Ticker(tickers[3]).history(
        start=startDate, end=endDate).reset_index()
    data = pd.DataFrame()
    all_assets = [s1, s2, s3, s4]

    for i, asset in enumerate(all_assets):
        asset = asset.reset_index()
        lb = tickers[i].lower()
        data[lb] = asset['Close']
    data['date'] = s1.Date
    return data
tickers1 = ["VTI", "AGG", "DBC", "^VIX"]
startDate = datetime.datetime(2010, 1, 1)
endDate = datetime.datetime(2023, 12, 1)
data = get_data( startDate, endDate,tickers1)
data

In [ ]:
tickers1 = ["VTI", "AGG", "DBC", "^VIX"]
startDate = datetime.datetime(2010, 1, 1)
endDate = datetime.datetime(2023, 12, 1)
data = get_data( startDate, endDate,tickers1)
data

In [ ]:
#prices = [vti.Close, agg.Close, dbc.Close, vix.Close]
prices_df = pd.DataFrame(data = {'date': vti.index,'vti':vti.Open.values, 'agg':agg.Open.values, 
                                 'dbc':dbc.Open.values, 'vix':vix.Open.values})
prices_df

In [ ]:
prices_df['vti_log_returns'] = np.log1p(prices_df.vti.pct_change())
prices_df['agg_log_returns'] = np.log1p(prices_df['agg'].pct_change())
prices_df['dbc_log_returns'] = np.log1p(prices_df.dbc.pct_change())
prices_df['vix_log_returns'] = np.log1p(prices_df.vix.pct_change())

In [ ]:
prices_df = prices_df.set_index(prices_df.date)
prices_df = prices_df.drop(columns = 'date')

In [ ]:
log_returns = prices_df.drop(columns = {'vti', 'agg', 'dbc', 'vix'})
log_returns

In [ ]:
log_returns

In [ ]:
returns_only = returns[['vti_return', 'agg_return', 'dbc_return', 'vix_return']]
returns_only

In [ ]:
#MVO experimentation

from scipy.optimize import minimize
def calculate_moving_average(df, window_size):
    return df.rolling(window=window_size).mean()

def mean_variance_optimization(returns_df):
    
    mean_returns = returns_df.mean()
    cov_matrix = returns_df.cov()
    num_assets = len(returns_df.columns)

    def neg(weights):
        return -np.sum(mean_returns * weights)
    
    weight_sum_constraint = (np.sum(weights) = 1)

    bounds = tuple((0, 1) for _ in range(num_assets))

    initial_weights = np.ones(num_assets) / num_assets

    result = minimize(neg, cov_matrix, initial_weights, method='SLSQP', bounds=bounds, constraints=[weight_sum_constraint])

    return result.x




In [ ]:
weights = mean_variance_optimization(returns_only[0:50])
weights

In [ ]:
cov_matrix = returns_only[0:51].cov()

In [ ]:
cov_matrix

In [ ]:

windows = log_returns.rolling(50, min_periods = 50)
weights_df = pd.DataFrame()

for i in windows:
    weights = mean_variance_optimization(i)
    weights_df = pd.concat([weights_df, pd.DataFrame([weights])], ignore_index=True)
    
    

In [ ]:
windows

In [ ]:
##Alternate simplified mvo approach with explicity variance minimizationt term:
def mean_variance_optimization(returns_df):

    returns = returns_df.mean()
    covariance = returns_df.cov()


    P = matrix(covariance.values)
    q = matrix(-returns)
    G = matrix(-np.identity(4))  # Negative identity matrix for inequality constraints
    h = matrix(np.zeros(4))  # Constraint: weights must be non-negative
    A = matrix(1.0, (1, 4))  # Constraint: sum of weights must equal 1
    b = matrix(1.0)


    solvers.options['show_progress'] = False
    solution = solvers.qp(P, q, G, h, A, b)


    weights = np.array(solution['x']).flatten()
    return weights


In [ ]:
returns_by_weight = pd.DataFrame(log_returns.values*weights_df.values, columns=returns_only.columns)

In [ ]:
returns_by_weight

In [ ]:
total_returns = returns_by_weight.sum(axis=1)
total_returns

In [ ]:
import matplotlib.pyplot as plt
plt.plot(total_returns.cumsum())

In [ ]:
windows = returns.drop(columns = ['date']).rolling(50)
weights_df = pd.DataFrame()

for i in windows:
    weights = mean_variance_optimization(i)
    weights_df = pd.concat([weights_df, pd.DataFrame([weights])], ignore_index=True)
    
    

In [ ]:
returns = returns.drop(columns = ['date'])
returns[1:]

In [ ]:
weighted_prices = pd.DataFrame(weights_df[:-1].values*returns[1:].values,
                               columns=returns[1:].columns)
weighted_prices

In [ ]:
total_daily_prices = weighted_prices.sum(axis = 1)
total_daily_prices

In [ ]:
total_returns = total_daily_prices.pct_change(1)
total_returns

In [ ]:
money = 1
money_arr = [money]
for i in total_returns[1:]:
    money = money + money * i
    money_arr.append(money)
    print(money)
money

In [ ]:
plot(money_arr)

In [ ]:
from matplotlib.pyplot import plot 

plot(total_returns)

In [ ]:
# So operate under the assumption of rebalance at close everyday. 

In [ ]:
#putting it all together:

from scipy.optimize import minimize

def get_data(startDate, endDate, tickers):
    s1 = yahooFinance.Ticker(tickers[0]).history(
        start=startDate, end=endDate).reset_index()
    s2 = yahooFinance.Ticker(tickers[1]).history(
        start=startDate, end=endDate).reset_index()
    s3 = yahooFinance.Ticker(tickers[2]).history(
        start=startDate, end=endDate).reset_index()
    s4 = yahooFinance.Ticker(tickers[3]).history(
        start=startDate, end=endDate).reset_index()
    data = pd.DataFrame()
    all_assets = [s1, s2, s3, s4]

    for i, asset in enumerate(all_assets):
        asset = asset.reset_index()
        lb = tickers[i].lower()
        data[lb] = asset['Close']

    return data


def mean_variance_optimization(returns_df):

    returns = returns_df.mean()
    covariance = returns_df.cov()


    P = matrix(covariance.values)
    q = matrix(-returns)
    G = matrix(-np.identity(4))  # Negative identity matrix for inequality constraints
    h = matrix(np.zeros(4))  # Constraint: weights must be non-negative
    A = matrix(1.0, (1, 4))  # Constraint: sum of weights must equal 1
    b = matrix(1.0)


    solvers.options['show_progress'] = False
    solution = solvers.qp(P, q, G, h, A, b)


    weights = np.array(solution['x']).flatten()
    return weights



In [ ]:
from matplotlib.pyplot import plot 

tickers1 = ["VTI", "AGG", "DBC", "^VIX"]
startDate = datetime.datetime(2010, 1, 1)
endDate = datetime.datetime(2021, 1, 1)
prices = get_data(startDate, endDate,tickers1)


windows = prices.rolling(50)
weights_df = pd.DataFrame()

for i in windows:
    try:
        weights = mean_variance_optimization(i)
        weights_df = pd.concat([weights_df, pd.DataFrame([weights])], ignore_index=True)
    except:
        pass


weighted_prices = pd.DataFrame(weights_df[:-1].values*prices[2:].values,
                               columns=prices[1:].columns)

total_daily_prices = weighted_prices.sum(axis = 1)
total_returns = total_daily_prices.pct_change(1)

money = 1
money_arr = [money]
for i in total_returns[1:]:
    money = money + money * i
    money_arr.append(money)



plot(range(len(prices[2:])),money_arr)

In [ ]:
weights_df = weights_df.rename(columns={"0": "vti", "1": "agg", '2':"dbc",})

In [ ]:
#Plotting
counter = 0
for i in weights_df.columns:
    plot(weights_df[i], label = prices.columns[counter])
    conuter = counter + 1

In [ ]:
plot(weights_df)

In [ ]:
portfolio_variance

In [ ]:
weights.value